In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Petfinding/FaceRecognition/

/content/drive/MyDrive/Petfinding/FaceRecognition


In [ ]:
!rm -R images

In [ ]:
!unzip /content/drive/MyDrive/Petfinding/FaceRecognition/catface_conf06.zip -d /content/drive/MyDrive/Petfinding/FaceRecognition

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/Petfinding/FaceRecognition/content/CutByV3/40f108d98-1.jpg  
  inflating: /content/drive/MyDrive/Petfinding/FaceRecognition/content/CutByV3/fb93de06f-3.jpg  
  inflating: /content/drive/MyDrive/Petfinding/FaceRecognition/content/CutByV3/98f5b7ecf-3.jpg  
  inflating: /content/drive/MyDrive/Petfinding/FaceRecognition/content/CutByV3/b751252cd-4.jpg  
  inflating: /content/drive/MyDrive/Petfinding/FaceRecognition/content/CutByV3/c4f05fe80-1.jpg  
  inflating: /content/drive/MyDrive/Petfinding/FaceRecognition/content/CutByV3/1a8fd6707-2.jpg  
  inflating: /content/drive/MyDrive/Petfinding/FaceRecognition/content/CutByV3/5ef34251f-4.jpg  
  inflating: /content/drive/MyDrive/Petfinding/FaceRecognition/content/CutByV3/a25a5ad9e-5.jpg  
  inflating: /content/drive/MyDrive/Petfinding/FaceRecognition/content/CutByV3/c2bb6a3be-3.jpg  
  inflating: /content/drive/MyDrive/Petfinding/FaceRecognition/content/CutBy

In [ ]:
!mv /content/drive/MyDrive/Petfinding/FaceRecognition/content/CutByV3/ /content/drive/MyDrive/Petfinding/FaceRecognition/
!mv CutByV3 images

In [ ]:
!rm -R /content/drive/MyDrive/Petfinding/FaceRecognition/images/CutByV3

rm: cannot remove '/content/drive/MyDrive/Petfinding/FaceRecognition/images/CutByV3': No such file or directory


In [ ]:
!rm -R embeddings
!mkdir embeddings

You probably saved the model using nn.DataParallel, which stores the model in module, and now you are trying to load it without DataParallel. You can either add a nn.DataParallel temporarily in your network for loading purposes, or you can load the weights file, create a new ordered dict without the module prefix, and load it back.

In [ ]:
!python get_embedding.py --weight "/content/drive/MyDrive/Petfinding/FaceRecognition/weights/bn_inception-239d2248.pth"

=> loading checkpoint /content/drive/MyDrive/Petfinding/FaceRecognition/weights/bn_inception-239d2248.pth
100% 440/440 [32:47<00:00,  4.47s/it]


Get_embedding: output to /embeddings dir. 

Transfer the images into embeddings, each image txt file contains two parts: ['image_path', 'embedding']

And save output embeddings to 'embeddings.pkl' 


# Prepare dataset

- spliting 70%/30% of classes to train/test sets reandomly (pets in the test sets are not seen in train sets): splitted by PetID

- one random image is extracted from each pet in the test set to form a query set. The rest images in the test set form a gallery set. We will use images in the query set to find matching images in the galerry set, and then calculate Recall@k for evaluation.

- print right images respectively

In [ ]:
!rm -R train
!rm -R test
!rm -R others
!mkdir train
!mkdir test
!mkdir others

In [ ]:
import glob
import os
class preprocessing():

  def __init__(self, path):
    self.path = path
  
  def read_all_txt(self):
    # if type_ != '.jpg' and type_ != '.png':
    #   raise Exception("Sorry, we need files of image in .jpg or .png")

    txtfiles = []
    address = self.path + '/*' + '.txt'
    for file in glob.glob(address):
      txtfiles.append(file)
    return txtfiles

  def read_all_file(self):
    # if type_ != '.jpg' and type_ != '.png':
    #   raise Exception("Sorry, we need files of image in .jpg or .png")

    txtfiles = []
    address = self.path + '/*' + '.jpg'
    for file in glob.glob(address):
      txtfiles.append(file)
    return txtfiles
  

  def get_petID(self):
    '''
    Input (list of string): filename with format: e.g. For 0fa6ba30b-1.jpg, 0fa6ba30b is the pet ID,  '1' is the number of image of given pet.

    Output: (list of string): list of PetIDs(labels)    
    '''
    txtfiles = self.read_all_txt()
    names = [x.split('/')[-1].split('-')[0] for x in txtfiles]
    return names


  def get_image_name(self):
    '''
    Input (list of string): filename with format: e.g. 0fa6ba30b-1.jpg

    Output: (list of string): list of PetIDs(labels)    
    '''
    txtfiles = self.read_all_file()
    image_path = [x.split('/')[-1].split('.')[0] for x in txtfiles]
    return image_path


  def load_image(self):
    img = tf.io.read_file(self.path)
    img_tensor = tf.image.decode_jpeg(img, channels= 3)
    return img_tensor

  def resize_normalize(self, img_size = 96):
    img = self.load_image()
    img = tf.image.resize(img, [img_size, img_size])
    img /= 255.0
    return img

In [ ]:
d = preprocessing("/content/drive/MyDrive/Petfinding/FaceRecognition/embeddings")
embeddinglist = d.read_all_txt()
embeddingID = d.get_petID()
print("There are total ", len(embeddingID), " of embedding images.")

There are total  14067  of embedding images.


In [ ]:
a = preprocessing("/content/drive/MyDrive/Petfinding/FaceRecognition/images")
PetID = a.get_petID()
filelist = a.read_all_file()
print("There is total ",len(filelist), " cats to be splitted into train/test sets.")

There is total  14067  cats to be splitted into train/test sets.


In [ ]:
from sklearn.model_selection import train_test_split
PetID_train, PetID_test = train_test_split(embeddingID, shuffle=True, test_size=0.30, random_state=42)
print("Train set of PetID length: ",len(PetID_train), " , test set of PetID: ",len(PetID_test))

Train set of PetID length:  9846  , test set of PetID:  4221


In [ ]:
import shutil
from os import path

trainset = set(PetID_train)
testset = set(PetID_test)

for i in embeddinglist:
  id = i.split('/')[-1].split('-')[0]
  source_path = i
  if id in trainset:
    new_location = shutil.move(source_path, "/content/drive/MyDrive/Petfinding/FaceRecognition/train")
  elif id in testset:
    new_location = shutil.move(source_path, "/content/drive/MyDrive/Petfinding/FaceRecognition/test")
  else:
    new_location = shutil.move(source_path, "/content/drive/MyDrive/Petfinding/FaceRecognition/others")

In [ ]:
!rm -R query
!rm -R gallery
!rm -R visualization
!mkdir query
!mkdir gallery
!mkdir visualization

In [ ]:
queryID = []
galleryID = []

b = preprocessing("/content/drive/MyDrive/Petfinding/FaceRecognition/test")
testlist = b.read_all_txt()
len(testlist)

105

In [ ]:
for i in testlist:
  id = i.split('/')[-1].split('-')[0]
  if id not in set(queryID):
    #move the first new id to query
    new_location = shutil.move(i, "/content/drive/MyDrive/Petfinding/FaceRecognition/query")
    queryID.append(id)
  else: #if id in set(queryID)
    new_location = shutil.move(i, "/content/drive/MyDrive/Petfinding/FaceRecognition/gallery")
    galleryID.append(i.split('/')[-1].split('.')[0])

In [ ]:
c = preprocessing("/content/drive/MyDrive/Petfinding/FaceRecognition/query")
querylist = c.read_all_txt()
print("There are total ", len(querylist), " of query images.")

There are total  33  of query images.


# Run Face Recog

In [ ]:
%cd /content/drive/MyDrive/Petfinding/FaceRecognition/
!python search.py --number 5 --save_path '/content/drive/MyDrive/Petfinding/FaceRecognition/Recog_result/R@5'

/content/drive/MyDrive/Petfinding/FaceRecognition


In [ ]:
!python search.py --number 50 --save_path '/content/drive/MyDrive/Petfinding/FaceRecognition/Recog_result/R@50'

In [ ]:
!python search.py --number 100 --save_path '/content/drive/MyDrive/Petfinding/FaceRecognition/Recog_result/R@100'

# Evaluation R@5

In [ ]:
import pickle
with open("/content/drive/MyDrive/Petfinding/FaceRecognition/Recog_result/R@5/search_results.pkl", 'rb') as f:
    data = pickle.load(f)

In [ ]:
data

,query_image_path,found_images_paths
0,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...
1,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...
2,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...
3,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...
4,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...
5,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...
6,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...
7,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...
8,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...
9,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...


In [ ]:
data["found_images_paths"][0]

['/content/drive/MyDrive/Petfinding/FaceRecognition/images/13966b6a8-5.jpg',
 '/content/drive/MyDrive/Petfinding/FaceRecognition/images/13966b6a8-1.jpg',
 '/content/drive/MyDrive/Petfinding/FaceRecognition/images/74f391caa-2.jpg',
 '/content/drive/MyDrive/Petfinding/FaceRecognition/images/c9307b1ee-3.jpg',
 '/content/drive/MyDrive/Petfinding/FaceRecognition/images/74f391caa-3.jpg']

In [ ]:
result = []
sum=0
for i in range(len(data["query_image_path"])):
  trueID = data["query_image_path"][i].split('/')[-1].split('-')[0]
  relevant=0
  for j in data["found_images_paths"][i]:
    predID = j.split('/')[-1].split('-')[0]
    if predID == trueID:
      relevant=relevant+1
  if relevant > 0:
    result.append(True)
    sum = sum+relevant
  else:
    result.append(False)

In [ ]:
#Recall@k = (# of recommended items @k that are relevant) / (total # of relevant items)
#R@k. It is a common metric in Deep Metric Learning and can be interpreted as the possibility that at least one photo in top k results picked by a model shows the target. 
count = 0
for i in result:
  if i == True:
    count+=1
  
recall = count / len(result)
recall

0.45454545454545453

In [ ]:
#Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)
Precision = sum /  (5*len(data["query_image_path"]))
Precision

0.1393939393939394

# Evaluation R@50

In [ ]:
import pickle
with open("/content/drive/MyDrive/Petfinding/FaceRecognition/Recog_result/R@50/search_results.pkl", 'rb') as f:
    data = pickle.load(f)

In [ ]:
data

,query_image_path,found_images_paths
0,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...
1,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...
2,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...
3,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...
4,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...
5,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...
6,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...
7,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...
8,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...
9,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...


In [ ]:
data["found_images_paths"][0]

['/content/drive/MyDrive/Petfinding/FaceRecognition/images/13966b6a8-5.jpg',
 '/content/drive/MyDrive/Petfinding/FaceRecognition/images/13966b6a8-1.jpg',
 '/content/drive/MyDrive/Petfinding/FaceRecognition/images/74f391caa-2.jpg',
 '/content/drive/MyDrive/Petfinding/FaceRecognition/images/c9307b1ee-3.jpg',
 '/content/drive/MyDrive/Petfinding/FaceRecognition/images/74f391caa-3.jpg',
 '/content/drive/MyDrive/Petfinding/FaceRecognition/images/6387fd0b7-2.jpg',
 '/content/drive/MyDrive/Petfinding/FaceRecognition/images/add2629da-2.jpg',
 '/content/drive/MyDrive/Petfinding/FaceRecognition/images/83166fea8-5.jpg',
 '/content/drive/MyDrive/Petfinding/FaceRecognition/images/c2bc9cfc3-2.jpg',
 '/content/drive/MyDrive/Petfinding/FaceRecognition/images/40713e16a-1.jpg',
 '/content/drive/MyDrive/Petfinding/FaceRecognition/images/ab17ce306-8.jpg',
 '/content/drive/MyDrive/Petfinding/FaceRecognition/images/c9307b1ee-1.jpg',
 '/content/drive/MyDrive/Petfinding/FaceRecognition/images/1b7d01262-1.jpg',

In [ ]:
result = []
sum=0
for i in range(len(data["query_image_path"])):
  trueID = data["query_image_path"][i].split('/')[-1].split('-')[0]
  relevant=0
  for j in data["found_images_paths"][i]:
    predID = j.split('/')[-1].split('-')[0]
    if predID == trueID:
      relevant=relevant+1
  if relevant > 0:
    result.append(True)
    sum = sum+relevant
  else:
    result.append(False)

In [ ]:
#Recall@k = (# of recommended items @k that are relevant) / (total # of relevant items)
#R@k. It is a common metric in Deep Metric Learning and can be interpreted as the possibility that at least one photo in top k results picked by a model shows the target. 
count = 0
for i in result:
  if i == True:
    count+=1
  
recall = count / len(result)
recall

1.0

In [ ]:
#Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)
Precision = sum /  (50*len(data["query_image_path"]))
Precision

0.040606060606060604

# Evaluation R@100

In [ ]:
import pickle
with open("/content/drive/MyDrive/Petfinding/FaceRecognition/Recog_result/R@100/search_results.pkl", 'rb') as f:
    data = pickle.load(f)

In [ ]:
data

,query_image_path,found_images_paths
0,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...
1,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...
2,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...
3,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...
4,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...
5,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...
6,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...
7,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...
8,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...
9,/content/drive/MyDrive/Petfinding/FaceRecognit...,[/content/drive/MyDrive/Petfinding/FaceRecogni...


In [ ]:
data["found_images_paths"][0]

['/content/drive/MyDrive/Petfinding/FaceRecognition/images/13966b6a8-5.jpg',
 '/content/drive/MyDrive/Petfinding/FaceRecognition/images/13966b6a8-1.jpg',
 '/content/drive/MyDrive/Petfinding/FaceRecognition/images/74f391caa-2.jpg',
 '/content/drive/MyDrive/Petfinding/FaceRecognition/images/c9307b1ee-3.jpg',
 '/content/drive/MyDrive/Petfinding/FaceRecognition/images/74f391caa-3.jpg',
 '/content/drive/MyDrive/Petfinding/FaceRecognition/images/6387fd0b7-2.jpg',
 '/content/drive/MyDrive/Petfinding/FaceRecognition/images/add2629da-2.jpg',
 '/content/drive/MyDrive/Petfinding/FaceRecognition/images/83166fea8-5.jpg',
 '/content/drive/MyDrive/Petfinding/FaceRecognition/images/c2bc9cfc3-2.jpg',
 '/content/drive/MyDrive/Petfinding/FaceRecognition/images/40713e16a-1.jpg',
 '/content/drive/MyDrive/Petfinding/FaceRecognition/images/ab17ce306-8.jpg',
 '/content/drive/MyDrive/Petfinding/FaceRecognition/images/c9307b1ee-1.jpg',
 '/content/drive/MyDrive/Petfinding/FaceRecognition/images/1b7d01262-1.jpg',

In [ ]:
result = []
sum=0
for i in range(len(data["query_image_path"])):
  trueID = data["query_image_path"][i].split('/')[-1].split('-')[0]
  relevant=0
  for j in data["found_images_paths"][i]:
    predID = j.split('/')[-1].split('-')[0]
    if predID == trueID:
      relevant=relevant+1
  if relevant > 0:
    result.append(True)
    sum = sum+relevant
  else:
    result.append(False)

In [ ]:
#Recall@k = (# of recommended items @k that are relevant) / (total # of relevant items)
#R@k. It is a common metric in Deep Metric Learning and can be interpreted as the possibility that at least one photo in top k results picked by a model shows the target. 
count = 0
for i in result:
  if i == True:
    count+=1
  
recall = count / len(result)
recall

1.0

In [ ]:
#Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)
Precision = sum / (100*len(data["query_image_path"]))
Precision

0.02181818181818182

**Citation:**

https://blog.csdn.net/golfbears/article/details/109387620

https://zhuanlan.zhihu.com/p/136384062

https://zhen8838.github.io/2020/06/17/proxy-anchor/

https://arxiv.org/pdf/2003.13911.pdf